In [1]:
import requests
import requests_cache
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter
from bs4 import BeautifulSoup
import time
import json
import re
import numpy as np
import pandas as pd # Dùng để đọc và hiển thị file csv/tsv
from datetime import datetime, timedelta # Dùng để xử lý dữ liệu thời gian

import urllib.robotparser # Kiểm tra file robot.txt có được phép crawl không

In [2]:
sleep_time = 1 # Đơn vị: giây 

In [3]:
ignoreRes = lambda s: '"incomplete_results":true' not in s.text
requests_cache.install_cache(expire_after=-1, filter_fn=ignoreRes)

In [4]:
def get_user_track(user_id_file, client_id):
    '''
    Thu thập dữ liệu các track của các user tương ứng trong user_id_file
    
    Parameters
    -------
    user_id_file : đường dẫn file .txt chứa danh sách user ID cần 
                   thu thập dữ liệu của các track
    client_id : ID truy cập API soundcloud
    
    Returns
    -------
    track : list of dictionary của các track thu được
    '''
    # Nạp user ID
    list_user_id = np.loadtxt(user_id_file, dtype=int)
    
    # Tạo list rỗng để lưu data thu được
    track = []
    
    # Cấu hình retry của request nếu gặp lỗi
    retries = Retry(total=6, backoff_factor=sleep_time, status_forcelist=[403, 500, 502, 503, 504 ])
    s = requests.Session()
    s.mount('https://', HTTPAdapter(max_retries=retries))

    try:
        # Duyết từng user ID
        for user_id in list_user_id:  
            # Lưu số lượng data đã thu được hiện tại
            # để sau này tránh được bot cài vào api
            checkpoint = len(track)

            # Lấy request với user_id tương ứng
            url = f'https://api-v2.soundcloud.com/users/{user_id}/tracks?client_id={client_id}&limit=100'
            
            # Ghi log để theo dõi
            print('User id:', user_id)
            print(' Get:', url)
            print('  Status: Doing')

            # Kiểm tra user này đã thu thập hết dữ liệu chưa bằng cách 
            # kiểm tra 'next_href' còn dẫn đến ta đến page khác của user không
            this_user_done = False
            while not this_user_done:
                try:
                    re = s.get(url)
                except Exception as e:
                    # Bỏ qua việc thu thập nếu gặp phải lỗi hoặc chờ quá lâu
                    if e != KeyboardInterrupt:
                        print("  Error or timeout! Status code:",re.status_code)
                        print("  Ignore this track")
                        continue

                # Lặp liên tục đến khi request được
                canGet = False
                while not canGet:
                    # Sleep rồi request lại
                    if not re.ok:
                        time.sleep(sleep_time)
                        re = requests.get(url)
                        print('  Status: Not ok')
                        continue

                    # Check request timeout
                    # Nếu requiest timeout, sleep rồi request lại
                    if re.status_code == 504:
                        print('  Status: error', re.status_code)
                        time.sleep(sleep_time)
                        re = requests.get(url)
                        continue
                    
                    # Vượt ngưỡng số lần cho phép của API -> dừng toàn bộ việc thu thập
                    if re.status_code == 429:
                        print('  Status: Exceeds an API rate limit')
                        return track

                    # Convert Json string thành python dictionary
                    json_pydata = json.loads(re.text)

                    # Request đã xong
                    canGet = True

                # In ra log
                print('  Status: Done')
                
                # Mở rộng tập dữ liệu vừa thu được
                track = track + json_pydata['collection']

                # Kiểm tra user này còn page khác không
                if json_pydata['next_href']:
                    # Vượt ngưỡng cho phép -> dừng thu thập user này
                    if len(track) - checkpoint >= 21892:
                        this_user_done = True
                        print(' # current tracks:', len(track))
                        continue

                    # Gán lại url page mới rồi tiếp thục thu thập
                    next_url = json_pydata['next_href']
                    url = next_url[:next_url.find('offset')] + f'client_id={client_id}&' + next_url[next_url.find('offset'):]
                    print(' Get:', url)
                    print('  Status: Doing')
                else:
                    this_user_done = True
                    print(' # current tracks:', len(track))
    except Exception as e:
        print(e)
    # Cho phép tạm ngưng thu thập dữ liệu nhưng có trả về tập dữ liệu hiện tại
    finally: 
        print('=========DONE=========')
        return track
        
    return track

In [5]:
track = get_user_track('user_id.txt', '3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn')

User id: 561697959
 Get: https://api-v2.soundcloud.com/users/561697959/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/561697959/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2019-12-30T16%3A56%3A06.000Z%2Ctracks%2C00735880021&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/561697959/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2019-04-08T16%3A00%3A09.000Z%2Ctracks%2C00602848785&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/561697959/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2019-01-10T18%3A47%3A27.000Z%2Ctracks%2C00556978200&limit=100
  Status: Doing
  Status: Done
 # current tracks: 239
User id: 169525608
 Get: https://api-v2.soundcloud.com/users/169525608/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/16952

  Status: Done
 Get: https://api-v2.soundcloud.com/users/227804586/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2016-06-16T18%3A00%3A43.000Z%2Ctracks%2C00264844127&limit=100
  Status: Doing
  Status: Done
 # current tracks: 582
User id: 498694473
 Get: https://api-v2.soundcloud.com/users/498694473/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/498694473/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2018-08-24T22%3A26%3A23.000Z%2Ctracks%2C00490224222&limit=100
  Status: Doing
  Status: Done
 # current tracks: 603
User id: 305017680
 Get: https://api-v2.soundcloud.com/users/305017680/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/305017680/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2018-09-02T19%3A54%3A23.000Z%2Ctracks%2C00494272611&limit=100
  Status: Doing
  Status: Done
 # curren

  Status: Done
 # current tracks: 1009
User id: 290616612
 Get: https://api-v2.soundcloud.com/users/290616612/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/290616612/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2017-02-22T12%3A30%3A57.000Z%2Ctracks%2C00308984969&limit=100
  Status: Doing
  Status: Done
 # current tracks: 1052
User id: 250740589
 Get: https://api-v2.soundcloud.com/users/250740589/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/250740589/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2018-09-02T17%3A32%3A24.000Z%2Ctracks%2C00494216118&limit=100
  Status: Doing
  Status: Done
 # current tracks: 1052
User id: 127805844
 Get: https://api-v2.soundcloud.com/users/127805844/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.so

  Status: Done
 Get: https://api-v2.soundcloud.com/users/250668/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2019-05-20T10%3A41%3A58.000Z%2Ctracks%2C00623532954&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/250668/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2019-04-11T09%3A14%3A53.000Z%2Ctracks%2C00604308054&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/250668/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2019-03-11T11%3A16%3A55.000Z%2Ctracks%2C00588274506&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/250668/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2019-01-29T14%3A05%3A16.000Z%2Ctracks%2C00566595738&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/250668/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2018-12-07T13%3A12%3A13.000Z%2Ctracks%2C00541141866&limit=100
  Status: Doing
  Sta

  Status: Done
 Get: https://api-v2.soundcloud.com/users/250668/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2011-03-08T00%3A00%3A00.000Z%2Ctracks%2C00223783482&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/250668/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2010-12-14T01%3A54%3A06.000Z%2Ctracks%2C00008029429&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/250668/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2010-09-28T18%3A28%3A20.000Z%2Ctracks%2C00005647218&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/250668/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2010-09-28T00%3A00%3A00.000Z%2Ctracks%2C00223795954&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/250668/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2010-09-28T00%3A00%3A00.000Z%2Ctracks%2C00223784134&limit=100
  Status: Doing
  Sta

...

  Status: Done
 Get: https://api-v2.soundcloud.com/users/3941306/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2013-07-04T13%3A47%3A51.000Z%2Ctracks%2C00099656260&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/3941306/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2013-04-24T18%3A04%3A55.000Z%2Ctracks%2C00089325255&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/3941306/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2013-02-01T11%3A57%3A09.000Z%2Ctracks%2C00077385097&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/3941306/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2012-10-30T17%3A41%3A08.000Z%2Ctracks%2C00065380869&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/3941306/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2012-08-22T14%3A35%3A20.000Z%2Ctracks%2C00057177993&limit=100
  Status: Doing


  Status: Done
 Get: https://api-v2.soundcloud.com/users/726617962/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2019-11-11T22%3A26%3A43.000Z%2Ctracks%2C00711336037&limit=100
  Status: Doing
  Status: Done
 # current tracks: 166199
User id: 1483778
 Get: https://api-v2.soundcloud.com/users/1483778/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/1483778/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2010-08-23T10%3A51%3A50.000Z%2Ctracks%2C00004792470&limit=100
  Status: Doing
  Status: Done
 # current tracks: 166219
User id: 37185433
 Get: https://api-v2.soundcloud.com/users/37185433/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/37185433/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2019-05-10T21%3A21%3A14.000Z%2Ctracks%2C00618769056&limit=100
  Status: Doing
  Status: Done
 Get: https:

  Status: Done
 # current tracks: 166482
User id: 11150457
 Get: https://api-v2.soundcloud.com/users/11150457/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/11150457/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2015-05-30T12%3A48%3A02.000Z%2Ctracks%2C00207961483&limit=100
  Status: Doing
  Status: Done
 # current tracks: 166483
User id: 132829477
 Get: https://api-v2.soundcloud.com/users/132829477/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/132829477/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2015-06-10T11%3A32%3A23.000Z%2Ctracks%2C00209675247&limit=100
  Status: Doing
  Status: Done
 # current tracks: 166490
User id: 392909973
 Get: https://api-v2.soundcloud.com/users/392909973/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2

  Status: Done
 # current tracks: 166781
User id: 949996546
 Get: https://api-v2.soundcloud.com/users/949996546/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/949996546/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2021-02-19T04%3A36%3A56.000Z%2Ctracks%2C00988579459&limit=100
  Status: Doing
  Status: Done
 # current tracks: 166782
User id: 452408079
 Get: https://api-v2.soundcloud.com/users/452408079/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/452408079/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2018-10-08T20%3A15%3A08.000Z%2Ctracks%2C00511451631&limit=100
  Status: Doing
  Status: Done
 # current tracks: 166790
User id: 30332696
 Get: https://api-v2.soundcloud.com/users/30332696/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v

  Status: Done
 Get: https://api-v2.soundcloud.com/users/196655211/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2020-02-27T20%3A31%3A10.000Z%2Ctracks%2C00767606140&limit=100
  Status: Doing
  Status: Done
 # current tracks: 167153
User id: 135747087
 Get: https://api-v2.soundcloud.com/users/135747087/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/135747087/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2017-05-25T05%3A54%3A23.000Z%2Ctracks%2C00324384177&limit=100
  Status: Doing
  Status: Done
 # current tracks: 167172
User id: 995654242
 Get: https://api-v2.soundcloud.com/users/995654242/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&limit=100
  Status: Doing
  Status: Done
 Get: https://api-v2.soundcloud.com/users/995654242/tracks?client_id=3jXdkVwgGnCwmB9q5e7qkzpaVm4qjQSn&offset=2021-06-14T14%3A07%3A13.000Z%2Ctracks%2C01068177499&limit=100
  Status: Doing
  Status: Done
 # 

In [6]:
# Danh sách các thuộc tính cần trích rút từ tập dữ liệu track
attributes = ['id', 'title', 'duration','full_duration', 'playback_count', 'likes_count', 'reposts_count',
                                   'comment_count', 'download_count', 'downloadable', 'commentable', 'has_downloads_left', 'state', 
                                    'genre', 'streamable', 'purchase_title', 'purchase_url', 'tag_list', 'visuals', 'user_id',
                                    'release_date', 'created_at', 'last_modified', 'license', 'policy', 'artwork_url', 'permalink_url']

In [7]:
# Biến list of dictionary thành DataFrame
df = pd.DataFrame(track)[attributes]

In [8]:
# Xuất file csv
df.to_csv('track.csv')

In [9]:
# Đọc thử file csv và xem đầu tập dữ liệu
df = pd.read_csv('track.csv')
df.head()

,Unnamed: 0,id,title,duration,full_duration,playback_count,likes_count,reposts_count,comment_count,download_count,...,tag_list,visuals,user_id,release_date,created_at,last_modified,license,policy,artwork_url,permalink_url
0,0,940057294,The Most Transparent Episode in History,4894589,4894589,NaN,NaN,0,NaN,NaN,...,NaN,NaN,561697959,NaN,2020-12-02T17:47:44Z,2020-12-02T17:51:14Z,all-rights-reserved,ALLOW,NaN,https://soundcloud.com/circling-back-podcast/t...
1,1,938810590,Alien Monoliths & Rating Will's Christmas Tree,4797518,4797518,NaN,NaN,0,NaN,NaN,...,NaN,NaN,561697959,NaN,2020-11-30T18:20:32Z,2020-11-30T18:20:50Z,all-rights-reserved,ALLOW,NaN,https://soundcloud.com/circling-back-podcast/a...
2,2,934649596,Dillon's Christmas Tree & Human Steaks,4746736,4746736,NaN,NaN,0,NaN,NaN,...,NaN,NaN,561697959,NaN,2020-11-23T18:10:53Z,2020-11-23T18:10:53Z,all-rights-reserved,ALLOW,NaN,https://soundcloud.com/circling-back-podcast/d...
3,3,931723405,Horny Popes & Dillon's Broken Toe,4264856,4264856,NaN,NaN,0,NaN,NaN,...,NaN,NaN,561697959,NaN,2020-11-18T20:59:42Z,2020-11-18T21:02:41Z,all-rights-reserved,ALLOW,NaN,https://soundcloud.com/circling-back-podcast/h...
4,4,930364729,Shattered Screens & Army Dolphins,4104307,4104307,NaN,NaN,0,NaN,NaN,...,NaN,NaN,561697959,NaN,2020-11-16T17:59:49Z,2020-11-16T18:01:30Z,all-rights-reserved,ALLOW,NaN,https://soundcloud.com/circling-back-podcast/s...
